In [39]:
from selenium import webdriver
from time import sleep
import threading
import re
from tqdm import tnrange, tqdm_notebook
import urllib 

In [52]:
items = pd.read_csv('data/item.csv')
items_detail= pd.read_csv('data/item_detail.csv')

In [53]:
len(items), len(items_detail)

(5180, 5180)

In [54]:
merged = pd.merge(items, items_detail, how='inner', on='id_item')

In [55]:
def getname(name, brand): 
    return name.replace(str(brand), '')

In [56]:
merged['item_name'] = merged.apply(lambda row: getname(row['name'], row['brand']), axis=1)

In [175]:
def get_comment(id_item, keyword, driver):
     
    ukey = urllib.parse.quote(keyword.encode('utf8'), '/:') 
    
    url = "https://msearch.shopping.naver.com/search/all.nhn?origQuery={}&pagingIndex=1&viewType=lst&sort=rel&showFilter=true&frm=NVSHSRC&selectedFilterTab=price&query={}".format(ukey,ukey)
       
    driver.get(url) 
    sleep(1)
     
    reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .grade') 
    rows = []
    if len(reviews) > 0: 
        for review in reviews:
            
            parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
            link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
            num = review.find_element_by_css_selector('.num').text
            cmts = review.find_element_by_css_selector('.help').text
            
            rows.append([id_item, link, num, cmts, keyword])
    else :
        reviews = driver.find_elements_by_css_selector('div.info_main .info_buy .piece') 
        
        revcnt = 0
        
        for review in reviews:
            if review.text.startswith('리뷰'): 
                cmts = review.text.split('리뷰')[1] 
                revcnt += 1 
                parent = review.find_element_by_xpath(".//ancestor::div[@class='info_main']")
                link = parent.find_element_by_css_selector('a.a_link').get_attribute('href')
                #print(link)
                rows.append([id_item, link, np.nan, cmts, keyword])
        
        if revcnt == 0:
            rows.append([id_item, np.nan, np.nan, np.nan, keyword])
    return rows

def get_comment_detail(url, driver):
    driver.get(url) 
    lis = driver.find_elements_by_css_selector('#_review_list li')
    print(len(lis))
    
    
def get_comment_group(items, names):
    driver =  webdriver.PhantomJS()
    for item, name in zip(items, names): 
         
        review_links = get_comment(item, name, driver)
        items_comments.extend(review_links)
    driver.close() 

In [176]:
!killall phantomjs

No matching processes belonging to you were found


In [177]:
items_comments = []
for cnt in range(1, 11): 
    start = 518 * (cnt-1)
    end = 518 * cnt-1
    items = merged.loc[start:end]['id_item'].values
    f_names = merged.loc[start:end]['fullname'].values
  
    th = threading.Thread(target=get_comment_group, args=(items,f_names, ))
    th.start()

In [197]:
review_info = pd.DataFrame(items_comments,columns=['id_item','link','point','cnt','keyword'])

In [199]:
!ls data

item.csv        item_detail.csv


In [200]:
review_info.to_csv('data/review_info.csv',index=False)

In [201]:
review_info[review_info['points']]

,id_item,link,point,cnt,keyword
0,28834,http://cr2.shopping.naver.com/adcr.nhn?x=4LGAr...,4.7,67,브이디엘 페스티벌 네일 듀오
1,27647,NaN,NaN,NaN,메이크업포에버 스타 파우더 #975
2,27561,NaN,NaN,NaN,에스쁘아 브론즈 페인팅 워터프루프 아이펜슬 (브룸스트리트)
3,19290,http://cr2.shopping.naver.com/adcr.nhn?x=HR86O...,4.7,16,바닐라코 투 아이즈 듀오 스퀘어드
4,32607,http://cr2.shopping.naver.com/adcr.nhn?x=5E%2B...,4.8,6,문샷 포맨 하이 컷 선크림
5,30761,NaN,NaN,NaN,안네마리보린 SOS 센서티브 컨센트레이트
6,27989,http://cr2.shopping.naver.com/adcr.nhn?x=t2J6R...,NaN,1,더페이스샵 러블리 믹스 파스텔 쿠션 블러셔 03 플럼쿠션
7,27989,http://cr2.shopping.naver.com/adcr.nhn?x=iMYqi...,NaN,18,더페이스샵 러블리 믹스 파스텔 쿠션 블러셔 03 플럼쿠션
8,27989,http://cr2.shopping.naver.com/adcr.nhn?x=cqBHc...,NaN,3,더페이스샵 러블리 믹스 파스텔 쿠션 블러셔 03 플럼쿠션
9,27989,http://cr2.shopping.naver.com/adcr.nhn?x=Ue4Lf...,NaN,1,더페이스샵 러블리 믹스 파스텔 쿠션 블러셔 03 플럼쿠션
